In [2]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import pandas as pd
from nltk.stem.snowball import SnowballStemmer
import string
from string import digits
nltk.download('punkt')
nltk.download('stopwords')
import xgboost as xgb
import numpy as np
import sklearn as sk
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import ensemble, metrics, model_selection, naive_bayes

[nltk_data] Downloading package punkt to /home/tnsuser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/tnsuser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/usr/local/lib/python3.4/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
from stop_words import get_stop_words

In [13]:
import gensim
model = gensim.models.Word2Vec(df_train['cleaned_text'], size=100)

In [14]:
w2v = dict(zip(model.wv.index2word, model.wv.syn0))







In [15]:
w2v_array_len = list(w2v.items())[0][1].shape[0]


In [5]:
# нулей не должно быть!


def sum_up_word2vec_array(clnd_text):
    
    total = None
    
    for word in clnd_text:
        if word in w2v:
            if total is None:
                total = w2v[word]
            else:
                total = np.add(total, w2v[word])
                
    if total is None:
        return np.zeros(w2v_array_len)
    else:
        return total
    

In [6]:
df_train = pd.read_csv("../data/train.csv")
df_test = pd.read_csv("../data/test.csv")
# 3 столбца - id, text, author
df_train.head(n=3)

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP


In [7]:
remove_digits = str.maketrans('', '', digits)
def tokenize_stem(file_text):
    #firstly let's apply nltk tokenization
    file_text = file_text.translate(remove_digits)
    
    tokens = nltk.word_tokenize(file_text)

    #let's delete punctuation symbols
    tokens = [i for i in tokens if ( i not in string.punctuation )]

    #deleting stop_words
    stop_words = set(stopwords.words('english')).union(set(get_stop_words('english')))
    tokens = [i for i in tokens if ( i not in stop_words )]
    
    # additional stop words removal
    

    #cleaning words
    stemmer = SnowballStemmer("english")
    
    tokens = [stemmer.stem(i) for i in tokens]

    return tokens

In [8]:
eng_stopwords = set(stopwords.words('english')).union(set(get_stop_words('english')))

In [9]:
def lexical_diversity(file_text):
    file_text = file_text.translate(remove_digits)
    try:
        tokens = nltk.word_tokenize(file_text)
    except:
        nltk.download('punkt')
        tokens = nltk.word_tokenize(file_text)
        

    #let's delete punctuation symbols
    tokens = [i for i in tokens if ( i not in string.punctuation )]

    #cleaning words
    stemmer = SnowballStemmer("english")
    
    tokens = [stemmer.stem(i) for i in tokens]

    return len(set(tokens))/len(tokens)

In [12]:
# вытаскиваем "значимые" слова

from sklearn.feature_extraction.text import TfidfVectorizer


raw_documents_authors = ['', '', '']


for index, row in df_train.iterrows():
    
    if row['author'] == 'EAP':
        raw_documents_authors[0] += row['cleaned_text_string'] + ' '
    elif row['author'] == 'HPL':
        raw_documents_authors[1] += row['cleaned_text_string'] + ' '
    else:
        raw_documents_authors[2] += row['cleaned_text_string'] + ' '
        

# удалим уникальные слова, не встречающиеся у других писателей

eap_only = set(raw_documents_authors[0].split(' ')) - set(raw_documents_authors[1].split(' ')) - set(raw_documents_authors[2].split(' '))
hpl_only = set(raw_documents_authors[1].split(' ')) - set(raw_documents_authors[0].split(' ')) - set(raw_documents_authors[2].split(' '))
msh_only = set(raw_documents_authors[2].split(' ')) - set(raw_documents_authors[0].split(' ')) - set(raw_documents_authors[1].split(' '))

unique_words = eap_only.union(hpl_only).union(msh_only)

tf = TfidfVectorizer(analyzer='word')
idf_matrix =  tf.fit_transform(raw_documents_authors)
feature_names = tf.get_feature_names()
# dictionary_word = dict(zip(feature_names, idf_matrix))

dense_idf = [i.todense() for i in idf_matrix]
print(dense_idf)

max_weighted_term = []

eap_dense_list = dense_idf[0].tolist()[0]
hpl_dense_list = dense_idf[1].tolist()[0]
mws_dense_list = dense_idf[2].tolist()[0]

for inum, i in enumerate(eap_dense_list):
    max_weighted_term.append(max(hpl_dense_list[inum], mws_dense_list[inum], 
                             i))

max_tf_dict = dict(zip(feature_names, max_weighted_term))



[matrix([[ 0.00054413,  0.00054413,  0.00108826, ...,  0.00054413,
          0.        ,  0.        ]]), matrix([[ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.0013942,
          0.0006971]]), matrix([[ 0.,  0.,  0., ...,  0.,  0.,  0.]])]


In [20]:

def extract_top_words(tfidfdict, numwrd):

    top_word_dict, min_value, min_key = {}, 99, ''
    

    for k, v in max_tf_dict.items():
        # print(top_word_dict.values())
        # print(v)
        if k not in unique_words and k not in eng_stopwords:
        
            if len(top_word_dict) < numwrd:
                top_word_dict[k] = v
                if v <= min_value:
                    min_key = k
            else:
                # print(v, min(list(top_word_dict.values())))
                if v > min(list(top_word_dict.values())) and k not in eng_stopwords:

                    min_value = min(top_word_dict.values())

                    for ky, va in top_word_dict.items():
                        if va == min_value:
                            min_key = ky

                    top_word_dict.pop(min_key)
                    top_word_dict[k] = v
                
    return top_word_dict



def count_topwords(target_df):

    for word in high_tf_idf_words_columns:
        
        # TODO: костыль, нужен, когда у нас уже есть такие столбцы
        # в датасете
#         try:
#             target_df = target_df.drop(word, 1)
#         except ValueError:
#             pass
        

        def count_numwords(collist):
            value = 0

            for wd in collist:
                if wd == word:
                    value += 1
            return value


        target_df[word] = target_df.cleaned_text.apply(count_numwords)
        

another_top_words_dict = extract_top_words(max_tf_dict, 80)
        

In [52]:
high_tf_idf_words_columns = list(another_top_words_dict.keys())

# count_topwords(df_train)


In [11]:
df_train['cleaned_text'] = df_train.text.apply(tokenize_stem)
df_train['cleaned_text_string'] = df_train.cleaned_text.apply(' '.join)
df_train.head(n=3)


,id,text,author,cleaned_text,cleaned_text_string
0,id26305,"This process, however, afforded me no means of...",EAP,"[this, process, howev, afford, mean, ascertain...",this process howev afford mean ascertain dimen...
1,id17569,It never once occurred to me that the fumbling...,HPL,"[it, never, occur, fumbl, might, mere, mistak]",it never occur fumbl might mere mistak
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP,"[in, left, hand, gold, snuff, box, caper, hill...",in left hand gold snuff box caper hill cut man...


In [111]:
df_test['cleaned_text'] = df_test.text.apply(tokenize_stem)


df_train['w2v_array'] = df_train.cleaned_text.apply(sum_up_word2vec_array)
df_test['w2v_array'] = df_test.cleaned_text.apply(sum_up_word2vec_array)

In [114]:
df_train_copy = df_train

In [118]:
len(df_train_copy)

19579

In [58]:


def create_w2v_columns(target_df):
    
    counter = 0

    for index, row in target_df.iterrows():
        counter += 1
        
#         if counter % 100 == 0:
#             print('another hundred processed')
        
        w2v_array = row['w2v_array']
        
        for elnum, el in enumerate(w2v_array):
            target_df['w2v_feature_' + str(elnum)] = el
        

# create_w2v_columns(df_train)

In [59]:
create_w2v_columns(df_test)
create_w2v_columns(df_train)

In [ ]:
create_w2v_columns(df_test)

In [38]:
df_train.columns.values

array(['id', 'text', 'author', 'cleaned_text', 'cleaned_text_string'], dtype=object)

In [47]:
df_train.columns.values

array(['id', 'text', 'author', 'cleaned_text', 'cleaned_text_string'], dtype=object)

In [53]:
df_train.columns.values

array(['id', 'text', 'author', 'cleaned_text', 'cleaned_text_string',
       'length', 'num_words', 'num_unique_words', 'num_punctuations',
       'num_words_upper', 'num_words_title', 'mean_word_len',
       'num_stopwords', 'mws_index', 'eap_index', 'hpl_index',
       'lexical_diversity', 'w2v_array'], dtype=object)

In [54]:
# df_train['length']=df_train['cleaned_text_string'].apply(len)
# df_train["num_words"] = df_train["text"].apply(lambda x: len(str(x).split()))
# df_train["num_unique_words"] = df_train["text"].apply(lambda x: len(set(str(x).split())))
# df_train["num_punctuations"] =df_train['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
# df_train["num_words_upper"] = df_train["text"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))
# df_train["num_words_title"] = df_train["text"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))
# df_train["mean_word_len"] = df_train["text"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
# df_train["num_stopwords"] = df_train["text"].apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))
# df_train['mws_index']=df_train['cleaned_text'].apply(ind_val_mws)/df_train['length']
# df_train['eap_index']=df_train['cleaned_text'].apply(ind_val_eap)/df_train['length']
# df_train['hpl_index']=df_train['cleaned_text'].apply(ind_val_hpl)/df_train['length']
# df_train['lexical_diversity'] = df_train.text.apply(lexical_diversity)
# df_train['w2v_array'] = df_train.cleaned_text.apply(sum_up_word2vec_array)
count_topwords(df_train)

df_train.head(n=3)

,id,text,author,cleaned_text,cleaned_text_string,length,num_words,num_unique_words,num_punctuations,num_words_upper,...,face,night,must,said,much,may,word,saw,year,feel
0,id26305,"This process, however, afforded me no means of...",EAP,"[this, process, howev, afford, mean, ascertain...",this process howev afford mean ascertain dimen...,145,41,35,7,2,...,0,0,0,0,0,0,0,0,0,0
1,id17569,It never once occurred to me that the fumbling...,HPL,"[it, never, occur, fumbl, might, mere, mistak]",it never occur fumbl might mere mistak,38,14,14,1,0,...,0,0,0,0,0,0,0,0,0,0
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP,"[in, left, hand, gold, snuff, box, caper, hill...",in left hand gold snuff box caper hill cut man...,116,36,32,5,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
df_hpl=df_train[df_train['author']=='HPL']
df_hpl.describe()

,length,num_words,num_unique_words,num_punctuations,num_words_upper,num_words_title,mean_word_len,num_stopwords,lexical_diversity,even,...,great,year,littl,whose,came,said,see,door,certain,found
count,5635.000000,5635.000000,5635.000000,5635.000000,5635.000000,5635.000000,5635.000000,5635.000000,5635.000000,5635.000000,...,5635.000000,5635.000000,5635.000000,5635.000000,5635.000000,5635.000000,5635.000000,5635.000000,5635.000000,5635.000000
mean,92.474357,27.799645,24.437977,3.206921,0.500266,2.334694,4.625193,13.150665,0.889980,0.041526,...,0.036025,0.028749,0.024135,0.030169,0.038332,0.024845,0.028394,0.027329,0.028571,0.033895
std,50.839074,14.123252,11.053739,2.108637,0.852313,2.041579,0.554917,6.916422,0.076026,0.200409,...,0.208825,0.171311,0.160270,0.183094,0.192013,0.159049,0.171370,0.176640,0.170821,0.183894
min,7.000000,4.000000,3.000000,1.000000,0.000000,0.000000,2.222222,0.000000,0.250000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,57.000000,18.000000,17.000000,2.000000,0.000000,1.000000,4.258065,8.000000,0.840000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,84.000000,26.000000,23.000000,3.000000,0.000000,2.000000,4.600000,12.000000,0.891892,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,117.000000,35.000000,30.000000,4.000000,1.000000,3.000000,4.961538,17.000000,0.947368,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,561.000000,147.000000,102.000000,28.000000,10.000000,39.000000,7.833333,63.000000,1.000000,2.000000,...,4.000000,2.000000,2.000000,2.000000,1.000000,2.000000,3.000000,4.000000,2.000000,2.000000


In [32]:
df_eap=df_train[df_train['author']=='EAP']
df_eap.describe()

,length,num_words,num_unique_words,num_punctuations,num_words_upper,num_words_title,mean_word_len,num_stopwords,lexical_diversity,even,...,great,year,littl,whose,came,said,see,door,certain,found
count,7900.000000,7900.000000,7900.000000,7900.000000,7900.000000,7900.000000,7900.000000,7900.000000,7900.000000,7900.000000,...,7900.000000,7900.000000,7900.000000,7900.000000,7900.000000,7900.000000,7900.000000,7900.000000,7900.000000,7900.000000
mean,80.893038,25.442405,21.894937,4.096329,0.553291,2.102405,4.644952,12.747595,0.886060,0.037468,...,0.032278,0.016203,0.035063,0.011646,0.016203,0.045063,0.018481,0.017722,0.013038,0.030253
std,59.749772,18.567706,13.727397,3.573788,0.892966,2.052241,0.631340,9.619779,0.097354,0.195826,...,0.190537,0.134984,0.195948,0.119568,0.129235,0.223892,0.143784,0.153253,0.119953,0.174950
min,5.000000,2.000000,2.000000,1.000000,0.000000,0.000000,2.000000,0.000000,0.333333,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,12.000000,12.000000,2.000000,0.000000,1.000000,4.250000,6.000000,0.821429,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,65.000000,21.000000,19.000000,3.000000,0.000000,1.000000,4.600000,10.000000,0.894737,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,106.000000,33.000000,29.000000,5.000000,1.000000,2.000000,5.000000,17.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,925.000000,267.000000,155.000000,71.000000,15.000000,43.000000,11.000000,135.000000,1.000000,2.000000,...,4.000000,2.000000,3.000000,3.000000,2.000000,6.000000,4.000000,4.000000,3.000000,2.000000


In [33]:
df_mws=df_train[df_train['author']=='MWS']
df_mws.describe()

,length,num_words,num_unique_words,num_punctuations,num_words_upper,num_words_title,mean_word_len,num_stopwords,lexical_diversity,even,...,great,year,littl,whose,came,said,see,door,certain,found
count,6044.000000,6044.000000,6044.000000,6044.000000,6044.000000,6044.000000,6044.000000,6044.000000,6044.000000,6044.000000,...,6044.000000,6044.000000,6044.000000,6044.000000,6044.000000,6044.000000,6044.000000,6044.000000,6044.000000,6044.000000
mean,85.267869,27.417273,23.544672,3.833719,0.751489,2.124255,4.598182,13.896923,0.883407,0.049305,...,0.017704,0.025645,0.019854,0.022998,0.019358,0.034414,0.024653,0.008107,0.004467,0.024156
std,71.372940,23.134440,14.925835,2.840625,1.203636,1.759572,0.561558,12.196599,0.086804,0.225507,...,0.141565,0.166251,0.144178,0.168612,0.137791,0.184112,0.168378,0.091508,0.066693,0.158844
min,3.000000,2.000000,2.000000,1.000000,0.000000,0.000000,2.666667,0.000000,0.398990,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,48.000000,15.000000,14.000000,2.000000,0.000000,1.000000,4.250000,7.000000,0.823529,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,73.000000,23.000000,21.000000,3.000000,0.000000,2.000000,4.560791,12.000000,0.885714,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,107.000000,34.000000,30.000000,5.000000,1.000000,3.000000,4.907156,18.000000,0.950000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2709.000000,861.000000,429.000000,59.000000,27.000000,46.000000,10.500000,437.000000,1.000000,3.000000,...,4.000000,3.000000,2.000000,3.000000,1.000000,2.000000,4.000000,2.000000,1.000000,2.000000


In [22]:
wordset=set()

In [23]:
#делаю сет со всеми словами
for i in df_train.index:
    wordset |= set(df_train['cleaned_text'][i])
wordlist=list(wordset)

In [24]:
#делаю фрейм со словами
df_word=pd.DataFrame(columns=["word", "mws", "eap", "hpl", "all"])
df_word["word"]=wordlist
df_word["mws"]=0
df_word["eap"]=0
df_word["hpl"]=0
df_word["all"]=0
df_word.head()

,word,mws,eap,hpl,all
0,equit,0,0,0,0
1,unobtrus,0,0,0,0
2,ottoman,0,0,0,0
3,oaken,0,0,0,0
4,harmoni,0,0,0,0


In [25]:
# как мы будем эту штуку правильнее делать (возможно это жуткий костыль), я хз
# сначала создаем словарь где ключ - уникальное слово, а значение - его порядковый номер
# затем создаем разреженную матрицу, которую заполняем в зависимости от порядковых номеров 
word_dict = {}



In [26]:
#делаю сет со всеми словами
# и сразу заготовку под шапку(потом увидишь зачем)
counter = 0
head = []

for wordlist in df_train['cleaned_text']:
    for word in wordlist:
        if word not in word_dict:
            head.append(word)
            word_dict[word] = counter
            counter += 1


In [27]:
# видоизменять колонки в pandas руками по одному значению в строке или столбце - очень плохая идея
# колонка это numpy.ndarray, а значит при каждой итерации она будет пересоздаваться
# что угробит производительность
# делаем значит так. считаем где сколько и где встречались отдельные слова, затем создаем строку за строкой для 
# каждого предложения

list_of_lists = []

for wordlist in df_train['cleaned_text']:
    row = [0 for i in range(len(word_dict))]
    for word in wordlist:
        row[word_dict[word]] += 1
    list_of_lists.append(row)



In [28]:
print(len(list_of_lists))

19579


In [29]:
# ... и для того чтобы посмотреть встречаемость того или иного слова по авторам добавим такую колонку

count_frame = pd.DataFrame(list_of_lists)
count_frame['author'] = df_train['author']



In [30]:
# ... и теперь нормальную шапку делаем

count_frame.columns = head + ['author']


In [31]:
print(count_frame.head())

   this  process  howev  afford  mean  ascertain  dimens  dungeon  i  might  \
0     1        1      1       1     1          1       1        1  2      1   
1     0        0      0       0     0          0       0        0  0      1   
2     0        0      0       0     0          0       0        0  0      0   
3     0        0      0       0     0          0       0        0  0      0   
4     0        0      0       0     0          0       0        0  0      0   

    ...    aegidus  burr  bentley  waltzer  binder  brusqueri  adriat  ancona  \
0   ...          0     0        0        0       0          0       0       0   
1   ...          0     0        0        0       0          0       0       0   
2   ...          0     0        0        0       0          0       0       0   
3   ...          0     0        0        0       0          0       0       0   
4   ...          0     0        0        0       0          0       0       0   

   agir  author  
0     0     EAP  
1 

In [32]:
# Пока объединим все, потом может быть будем использовать
col=list(count_frame.columns)
col[-1]='author_name'
count_frame.columns=col
pivot_col=pd.pivot_table(count_frame, aggfunc=np.sum, values=col, index=['author_name'])

In [33]:
#Убираем лишние слова, которые не учли раньше
col=list(pivot_col.columns)
col2=[string for string in col if (string[0]!='"' and string[0]!="'"
                                  and string[0]!='.' and string[0]!='`'
                                   and len(string)>3 and '.' not in string)]
col=[]
pivot_col=pivot_col[col2]
pivot_col.head()

,aaem,aback,abaft,abandon,abaout,abas,abash,abat,abbey,abbrevi,...,æmilianus,æneid,ærial,æronaut,ærostat,æschylus,élite,émeut,οἶδα,υπνος
author_name,,,,,,,,,,,,,,,,,,,,,
EAP,1,2,0,22,0,2,1,2,3,2,...,0,0,1,3,1,1,1,1,0,0
HPL,0,0,0,17,24,0,1,3,0,0,...,2,1,0,0,0,0,0,0,2,1
MWS,0,0,1,9,0,0,0,1,2,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
#Create pivot
pivot_col=pivot_col.append(pivot_col.sum(), ignore_index=True)
pivot_col.index=['EAP', 'HPL', 'MWS', 'SUMA']
pivot_col.head()

,aaem,aback,abaft,abandon,abaout,abas,abash,abat,abbey,abbrevi,...,æmilianus,æneid,ærial,æronaut,ærostat,æschylus,élite,émeut,οἶδα,υπνος
EAP,1,2,0,22,0,2,1,2,3,2,...,0,0,1,3,1,1,1,1,0,0
HPL,0,0,0,17,24,0,1,3,0,0,...,2,1,0,0,0,0,0,0,2,1
MWS,0,0,1,9,0,0,0,1,2,0,...,0,0,0,0,0,0,0,0,0,0
SUMA,1,2,1,48,24,2,2,6,5,2,...,2,1,1,3,1,1,1,1,2,1


In [35]:
summa=[pivot_col.loc['EAP'].sum(), pivot_col.loc['HPL'].sum(), 
       pivot_col.loc['MWS'].sum(), pivot_col.loc['SUMA'].sum()]
pivot_col['summa']=summa
pivot_col.head()

,aaem,aback,abaft,abandon,abaout,abas,abash,abat,abbey,abbrevi,...,æneid,ærial,æronaut,ærostat,æschylus,élite,émeut,οἶδα,υπνος,summa
EAP,1,2,0,22,0,2,1,2,3,2,...,0,1,3,1,1,1,1,0,0,86909
HPL,0,0,0,17,24,0,1,3,0,0,...,1,0,0,0,0,0,0,2,1,73404
MWS,0,0,1,9,0,0,0,1,2,0,...,0,0,0,0,0,0,0,0,0,72297
SUMA,1,2,1,48,24,2,2,6,5,2,...,1,1,3,1,1,1,1,2,1,232610


In [36]:
# Create probability of author text knowing that a word was used
pivot_part=pivot_col
pivot_part.loc['EAP']=pivot_col.loc['EAP']/pivot_col.loc['SUMA']
pivot_part.loc['HPL']=pivot_col.loc['HPL']/pivot_col.loc['SUMA']
pivot_part.loc['MWS']=pivot_col.loc['MWS']/pivot_col.loc['SUMA']
pivot_part=pivot_part.loc[['EAP', 'HPL', 'MWS']]
# Delete unique words
pivot_part=pivot_part.loc[:, (pivot_part!=1).all(axis=0)]
pivot_part.head()

,abandon,abash,abat,abbey,abdic,aberr,abhor,abhorr,abil,abject,...,younger,youngest,your,youth,zeal,zenith,zest,zigzag,zone,summa
EAP,0.458333,0.5,0.333333,0.6,0.142857,0.166667,0.058824,0.111111,0.789474,0.333333,...,0.272727,0.2,0.534884,0.101562,0.117647,0.4,0.2,0.4,0.666667,0.373625
HPL,0.354167,0.5,0.500000,0.0,0.000000,0.666667,0.235294,0.555556,0.052632,0.000000,...,0.000000,0.4,0.069767,0.429688,0.470588,0.6,0.2,0.6,0.333333,0.315567
MWS,0.187500,0.0,0.166667,0.4,0.857143,0.166667,0.705882,0.333333,0.157895,0.666667,...,0.727273,0.4,0.395349,0.468750,0.411765,0.0,0.6,0.0,0.000000,0.310808


In [50]:
# It will be easier to work this way
eap_dict=pivot_part.loc['EAP'].to_dict()
hpl_dict=pivot_part.loc['HPL'].to_dict()
mws_dict=pivot_part.loc['MWS'].to_dict()
eap_dict['word']

0.44859813084112149

In [21]:
# Create author score 
def ind_val_eap(listn):
    quant=0
    for word in listn:
        try:
            quant+=eap_dict[word]
        except KeyError:
            quant+=0
    return quant

def ind_val_hpl(listn):
    quant=0
    for word in listn:
        try:
            quant+=hpl_dict[word]
        except KeyError:
            quant+=0
    return quant

def ind_val_mws(listn):
    quant=0
    for word in listn:
        try:
            quant+=mws_dict[word]
        except KeyError:
            quant+=0
    return quant

In [51]:
# Add index of author
df_train['mws_index']=df_train['cleaned_text'].apply(ind_val_mws)/df_train['length']
df_train['eap_index']=df_train['cleaned_text'].apply(ind_val_eap)/df_train['length']
df_train['hpl_index']=df_train['cleaned_text'].apply(ind_val_hpl)/df_train['length']
df_train.head(n=3)

,id,text,author,cleaned_text,cleaned_text_string,length,num_words,num_unique_words,num_punctuations,num_words_upper,...,whose,came,said,see,door,certain,found,mws_index,eap_index,hpl_index
0,id26305,"This process, however, afforded me no means of...",EAP,"[this, process, howev, afford, mean, ascertain...",this process howev afford mean ascertain dimen...,145,41,35,7,2,...,0,0,0,0,0,0,0,0.035935,0.074388,0.034504
1,id17569,It never once occurred to me that the fumbling...,HPL,"[it, never, occur, fumbl, might, mere, mistak]",it never occur fumbl might mere mistak,38,14,14,1,0,...,0,0,0,0,0,0,0,0.035860,0.060980,0.034739
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP,"[in, left, hand, gold, snuff, box, caper, hill...",in left hand gold snuff box caper hill cut man...,116,36,32,5,0,...,0,0,0,0,0,0,0,0.026900,0.047832,0.037337


In [52]:
#Transform authors' names to numeric
df_train['author']=df_train['author'].astype('category')
df_train['author2']=df_train['author'].cat.codes
# Create different features 
df_train.head(n=3)
mid = df_train['author2']
df_train.drop(labels=['author2'], axis=1,inplace = True)
df_train.insert(0, 'author2', mid)
df_train.head()

,author2,id,text,author,cleaned_text,cleaned_text_string,length,num_words,num_unique_words,num_punctuations,...,whose,came,said,see,door,certain,found,mws_index,eap_index,hpl_index
0,0,id26305,"This process, however, afforded me no means of...",EAP,"[this, process, howev, afford, mean, ascertain...",this process howev afford mean ascertain dimen...,145,41,35,7,...,0,0,0,0,0,0,0,0.035935,0.074388,0.034504
1,1,id17569,It never once occurred to me that the fumbling...,HPL,"[it, never, occur, fumbl, might, mere, mistak]",it never occur fumbl might mere mistak,38,14,14,1,...,0,0,0,0,0,0,0,0.035860,0.060980,0.034739
2,0,id11008,"In his left hand was a gold snuff box, from wh...",EAP,"[in, left, hand, gold, snuff, box, caper, hill...",in left hand gold snuff box caper hill cut man...,116,36,32,5,...,0,0,0,0,0,0,0,0.026900,0.047832,0.037337
3,2,id27763,How lovely is spring As we looked from Windsor...,MWS,"[how, love, spring, as, look, windsor, terrac,...",how love spring as look windsor terrac sixteen...,144,34,32,4,...,0,0,0,0,0,0,0,0.071850,0.033438,0.033601
4,1,id12958,"Finding nothing else, not even gold, the Super...",HPL,"[find, noth, els, even, gold, superintend, aba...",find noth els even gold superintend abandon at...,102,27,25,4,...,0,0,0,0,0,0,0,0.036859,0.056661,0.043735


In [63]:
author_mapping_dict = {'EAP':0, 'HPL':1, 'MWS':2}
train_y = df_train['author'].map(author_mapping_dict)
train_id = df_train['id'].values
test_id = df_test['id'].values
cols_to_drop = ['id', 'text']
train_X = df_train.drop(cols_to_drop+['author'], axis=1)
test_X = df_test.drop(cols_to_drop, axis=1)
tfidf_vec = TfidfVectorizer(stop_words='english', ngram_range=(1,3))
full_tfidf = tfidf_vec.fit_transform(df_train['text'].values.tolist() + df_test['text'].values.tolist())
train_tfidf = tfidf_vec.transform(df_train['text'].values.tolist())
test_tfidf = tfidf_vec.transform(df_test['text'].values.tolist())

In [64]:
def runMNB(train_X, train_y, test_X, test_y, test_X2):
    model = naive_bayes.MultinomialNB()
    model.fit(train_X, train_y)
    pred_test_y = model.predict_proba(test_X)
    pred_test_y2 = model.predict_proba(test_X2)
    return pred_test_y, pred_test_y2, model

In [65]:
cv_scores = []
pred_full_test = 0
pred_train = np.zeros([df_train.shape[0], 3])
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)
for dev_index, val_index in kf.split(train_X):
    dev_X, val_X = train_tfidf[dev_index], train_tfidf[val_index]
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    pred_val_y, pred_test_y, model = runMNB(dev_X, dev_y, val_X, val_y, test_tfidf)
    pred_full_test = pred_full_test + pred_test_y
    pred_train[val_index,:] = pred_val_y
    cv_scores.append(metrics.log_loss(val_y, pred_val_y))
print("Mean cv score : ", np.mean(cv_scores))
pred_full_test = pred_full_test / 5.

Mean cv score :  0.842216198361


In [66]:
n_comp = 20
svd_obj = TruncatedSVD(n_components=n_comp, algorithm='arpack')
svd_obj.fit(full_tfidf)
train_svd = pd.DataFrame(svd_obj.transform(train_tfidf))
test_svd = pd.DataFrame(svd_obj.transform(test_tfidf))
    
train_svd.columns = ['svd_word_'+str(i) for i in range(n_comp)]
test_svd.columns = ['svd_word_'+str(i) for i in range(n_comp)]
df_train = pd.concat([df_train, train_svd], axis=1)
df_test = pd.concat([df_test, test_svd], axis=1)
del full_tfidf, train_tfidf, test_tfidf, train_svd, test_svd

In [67]:
### Fit transform the count vectorizer ###
tfidf_vec = CountVectorizer(stop_words='english', ngram_range=(1,3))
tfidf_vec.fit(df_train['text'].values.tolist() + df_test['text'].values.tolist())
train_tfidf = tfidf_vec.transform(df_train['text'].values.tolist())
test_tfidf = tfidf_vec.transform(df_test['text'].values.tolist())

In [68]:
cv_scores = []
pred_full_test = 0
pred_train = np.zeros([df_train.shape[0], 3])
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)
for dev_index, val_index in kf.split(train_X):
    dev_X, val_X = train_tfidf[dev_index], train_tfidf[val_index]
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    pred_val_y, pred_test_y, model = runMNB(dev_X, dev_y, val_X, val_y, test_tfidf)
    pred_full_test = pred_full_test + pred_test_y
    pred_train[val_index,:] = pred_val_y
    cv_scores.append(metrics.log_loss(val_y, pred_val_y))
print("Mean cv score : ", np.mean(cv_scores))
pred_full_test = pred_full_test / 5.

# add the predictions as new features #
df_train["nb_cvec_eap"] = pred_train[:,0]
df_train["nb_cvec_hpl"] = pred_train[:,1]
df_train["nb_cvec_mws"] = pred_train[:,2]
df_test["nb_cvec_eap"] = pred_full_test[:,0]
df_test["nb_cvec_hpl"] = pred_full_test[:,1]
df_test["nb_cvec_mws"] = pred_full_test[:,2]

Mean cv score :  0.450918416166


In [69]:
print(X_train.shape)
print(x_t.shape)

NameError: name 'X_train' is not defined

In [70]:
# Create data set
ds_train=df_train.values
X=ds_train[:, 6:]
Y=ds_train[:, 0]
seed=7
test_size=0.3
X_train, X_test, y_train, y_test = sk.model_selection.train_test_split(X,Y, test_size=test_size, random_state=seed)

In [78]:
X_train = [[float(j) for j in i] for i in X_train]

In [79]:
xg_train=xgb.DMatrix(X_train, label=y_train)

TypeError: a float is required

In [71]:
# fit model
xg_train=xgb.DMatrix(X_train, label=y_train)
xg_test=xgb.DMatrix(X_test, label=y_test)
xg_t=xgb.DMatrix(X, label=Y)
param={}
param['objective'] = 'multi:softmax'
param['eta'] = 0.3
param['max_depth'] = 3
param['silent'] = 1
param['num_class'] = 3
param['eval_metric']= "mlogloss"
watchlist = [(xg_train, 'train'), (xg_test, 'test')]
num_round = 70
bst = xgb.train(param, xg_train, num_round, watchlist)
# get prediction
pred = bst.predict(xg_test)
error_rate = np.sum(pred != y_test) / y_test.shape[0]
print('Test error using softmax = {}'.format(error_rate))

TypeError: a float is required

In [141]:
# do the same thing again, but output probabilities
param['objective'] = 'multi:softprob'
bstp = xgb.train(param, xg_train, num_round, watchlist)
# Note: this convention has been changed since xgboost-unity
# get prediction, this is in 1D array, need reshape to (ndata, nclass)
pred_prob = bstp.predict(xg_test).reshape(y_test.shape[0], 3)
pred_label = np.argmax(pred_prob, axis=1)
error_rate = np.sum(pred_label != y_test) / y_test.shape[0]
print('Test error using softprob = {}'.format(error_rate))

[0]	train-mlogloss:0.835585	test-mlogloss:0.837725
[1]	train-mlogloss:0.68415	test-mlogloss:0.687706
[2]	train-mlogloss:0.587061	test-mlogloss:0.592043
[3]	train-mlogloss:0.521673	test-mlogloss:0.527715
[4]	train-mlogloss:0.476203	test-mlogloss:0.483356
[5]	train-mlogloss:0.443035	test-mlogloss:0.451809
[6]	train-mlogloss:0.419353	test-mlogloss:0.429201
[7]	train-mlogloss:0.400974	test-mlogloss:0.412195
[8]	train-mlogloss:0.387214	test-mlogloss:0.40011
[9]	train-mlogloss:0.375655	test-mlogloss:0.39004
[10]	train-mlogloss:0.366574	test-mlogloss:0.382388
[11]	train-mlogloss:0.358829	test-mlogloss:0.376405
[12]	train-mlogloss:0.352795	test-mlogloss:0.371874
[13]	train-mlogloss:0.347295	test-mlogloss:0.367584
[14]	train-mlogloss:0.342647	test-mlogloss:0.36442
[15]	train-mlogloss:0.338423	test-mlogloss:0.361727
[16]	train-mlogloss:0.334811	test-mlogloss:0.359318
[17]	train-mlogloss:0.331139	test-mlogloss:0.35714
[18]	train-mlogloss:0.328364	test-mlogloss:0.355447
[19]	train-mlogloss:0.32530

In [142]:
# do the same thing again, but output probabilities
param['objective'] = 'multi:softprob'
bstp = xgb.train(param, xg_t, num_round, watchlist)
# Note: this convention has been changed since xgboost-unity
# get prediction, this is in 1D array, need reshape to (ndata, nclass)
pred_prob = bstp.predict(xg_t).reshape(Y.shape[0], 3)
pred_label = np.argmax(pred_prob, axis=1)
error_rate = np.sum(pred_label != Y) / Y.shape[0]
print('Test error using softprob = {}'.format(error_rate))

[0]	train-mlogloss:0.836322	test-mlogloss:0.835926
[1]	train-mlogloss:0.685431	test-mlogloss:0.684692
[2]	train-mlogloss:0.588655	test-mlogloss:0.587662
[3]	train-mlogloss:0.523748	test-mlogloss:0.522269
[4]	train-mlogloss:0.478684	test-mlogloss:0.476855
[5]	train-mlogloss:0.44625	test-mlogloss:0.444233
[6]	train-mlogloss:0.423007	test-mlogloss:0.42076
[7]	train-mlogloss:0.404697	test-mlogloss:0.402665
[8]	train-mlogloss:0.391392	test-mlogloss:0.389223
[9]	train-mlogloss:0.380203	test-mlogloss:0.378347
[10]	train-mlogloss:0.371416	test-mlogloss:0.369665
[11]	train-mlogloss:0.36354	test-mlogloss:0.362414
[12]	train-mlogloss:0.357645	test-mlogloss:0.356544
[13]	train-mlogloss:0.352108	test-mlogloss:0.351681
[14]	train-mlogloss:0.347992	test-mlogloss:0.347653
[15]	train-mlogloss:0.343259	test-mlogloss:0.343469
[16]	train-mlogloss:0.33965	test-mlogloss:0.340181
[17]	train-mlogloss:0.336416	test-mlogloss:0.33691
[18]	train-mlogloss:0.333316	test-mlogloss:0.334061
[19]	train-mlogloss:0.33014

In [154]:
xg_train

In [144]:
df_test.head(n=3)

,id,text,length,num_words,num_unique_words,num_punctuations,num_words_upper,num_words_title,mean_word_len,num_stopwords,...,svd_word_10,svd_word_11,svd_word_12,svd_word_13,svd_word_14,svd_word_15,svd_word_16,svd_word_17,svd_word_18,svd_word_19
0,id02310,"Still, as I urged our leaving Ireland with suc...",67,19,19,3,1,3,4.842105,9,...,0.007403,-0.000670,-0.008355,-0.011837,0.036064,-0.016591,-0.025580,-0.018785,0.031289,-0.047220
1,id24541,"If a fire wanted fanning, it could readily be ...",175,62,49,7,1,3,4.338710,34,...,-0.045866,-0.017172,0.019988,-0.004397,-0.000020,-0.008583,0.006335,-0.004216,0.001810,0.001767
2,id00134,And when they had broken down the frail door t...,114,33,30,3,0,1,4.757576,15,...,-0.002360,0.002929,-0.017479,0.006063,-0.003324,-0.009452,0.013239,0.004852,-0.007478,0.002786


In [45]:
df_test['cleaned_text'] = df_test.text.apply(tokenize_stem)
df_test['cleaned_text_string'] = df_test.cleaned_text.apply(' '.join)
df_test.head(n=3)

,id,text,cleaned_text,cleaned_text_string
0,id02310,"Still, as I urged our leaving Ireland with suc...","[still, i, urg, leav, ireland, inquietud, impa...",still i urg leav ireland inquietud impati fath...
1,id24541,"If a fire wanted fanning, it could readily be ...","[if, fire, want, fan, readili, fan, newspap, g...",if fire want fan readili fan newspap govern gr...
2,id00134,And when they had broken down the frail door t...,"[and, broken, frail, door, found, two, clean, ...",and broken frail door found two clean pick hum...


In [60]:
df_test.columns.values

array(['id', 'text', 'cleaned_text', 'cleaned_text_string', 'length',
       'num_words', 'num_unique_words', 'num_punctuations',
       'num_words_upper', 'num_words_title', 'mean_word_len',
       'num_stopwords', 'mws_index', 'eap_index', 'hpl_index',
       'lexical_diversity', 'w2v_array', 'heard', 'raymond', 'fear',
       'long', 'chang', 'death', 'love', 'certain', 'like', 'man', 'come',
       'even', 'still', 'place', 'hand', 'door', 'strang', 'room', 'men',
       'friend', 'appear', 'father', 'eye', 'window', 'street', 'look',
       'us', 'made', 'well', 'die', 'ever', 'found', 'two', 'know',
       'never', 'say', 'thought', 'light', 'old', 'see', 'whose', 'thus',
       'happi', 'everi', 'dream', 'mani', 'time', 'seem', 'hope', 'upon',
       'natur', 'might', 'life', 'yet', 'though', 'mind', 'first', 'hous',
       'dark', 'heart', 'howev', 'day', 'one', 'return', 'pass', 'shall',
       'thing', 'came', 'great', 'littl', 'face', 'night', 'must', 'said',
       'much', 

In [61]:
df_train.columns.values

array(['id', 'text', 'author', 'cleaned_text', 'cleaned_text_string',
       'length', 'num_words', 'num_unique_words', 'num_punctuations',
       'num_words_upper', 'num_words_title', 'mean_word_len',
       'num_stopwords', 'mws_index', 'eap_index', 'hpl_index',
       'lexical_diversity', 'w2v_array', 'heard', 'raymond', 'fear',
       'long', 'chang', 'death', 'love', 'certain', 'like', 'man', 'come',
       'even', 'still', 'place', 'hand', 'door', 'strang', 'room', 'men',
       'friend', 'appear', 'father', 'eye', 'window', 'street', 'look',
       'us', 'made', 'well', 'die', 'ever', 'found', 'two', 'know',
       'never', 'say', 'thought', 'light', 'old', 'see', 'whose', 'thus',
       'happi', 'everi', 'dream', 'mani', 'time', 'seem', 'hope', 'upon',
       'natur', 'might', 'life', 'yet', 'though', 'mind', 'first', 'hous',
       'dark', 'heart', 'howev', 'day', 'one', 'return', 'pass', 'shall',
       'thing', 'came', 'great', 'littl', 'face', 'night', 'must', 'said',
     

In [55]:

df_test['length']=df_test['cleaned_text_string'].apply(len)
df_test["num_words"] = df_test["text"].apply(lambda x: len(str(x).split()))
df_test["num_unique_words"] = df_test["text"].apply(lambda x: len(set(str(x).split())))
df_test["num_punctuations"] =df_test['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
df_test["num_words_upper"] = df_test["text"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))
df_test["num_words_title"] = df_test["text"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))
df_test["mean_word_len"] = df_test["text"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
df_test["num_stopwords"] = df_test["text"].apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))
df_test['mws_index']=df_test['cleaned_text'].apply(ind_val_mws)/df_test['length']
df_test['eap_index']=df_test['cleaned_text'].apply(ind_val_eap)/df_test['length']
df_test['hpl_index']=df_test['cleaned_text'].apply(ind_val_hpl)/df_test['length']
df_test['lexical_diversity'] = df_test.text.apply(lexical_diversity)
df_test['w2v_array'] = df_test.cleaned_text.apply(sum_up_word2vec_array)
count_topwords(df_test)
df_test.head(n=3)

,id,text,cleaned_text,cleaned_text_string,length,num_words,num_unique_words,num_punctuations,num_words_upper,num_words_title,...,face,night,must,said,much,may,word,saw,year,feel
0,id02310,"Still, as I urged our leaving Ireland with suc...","[still, i, urg, leav, ireland, inquietud, impa...",still i urg leav ireland inquietud impati fath...,67,19,19,3,1,3,...,0,0,0,0,0,0,0,0,0,0
1,id24541,"If a fire wanted fanning, it could readily be ...","[if, fire, want, fan, readili, fan, newspap, g...",if fire want fan readili fan newspap govern gr...,175,62,49,7,1,3,...,0,0,0,0,0,0,0,0,0,0
2,id00134,And when they had broken down the frail door t...,"[and, broken, frail, door, found, two, clean, ...",and broken frail door found two clean pick hum...,114,33,30,3,0,1,...,0,0,0,0,0,0,0,0,0,0


In [67]:
df_test['mws_index']=df_test['cleaned_text'].apply(ind_val_mws)/df_test['length']
df_test['eap_index']=df_test['cleaned_text'].apply(ind_val_eap)/df_test['length']
df_test['hpl_index']=df_test['cleaned_text'].apply(ind_val_hpl)/df_test['length']
df_test.head(n=3)

,id,text,cleaned_text,cleaned_text_string,length,num_words,num_unique_words,num_punctuations,num_words_upper,num_words_title,...,whose,came,said,see,door,certain,found,mws_index,eap_index,hpl_index
0,id02310,"Still, as I urged our leaving Ireland with suc...","[still, i, urg, leav, ireland, inquietud, impa...",still i urg leav ireland inquietud impati fath...,67,19,19,3,1,3,...,0,0,0,0,0,0,0,0.071473,0.036315,0.026541
1,id24541,"If a fire wanted fanning, it could readily be ...","[if, fire, want, fan, readili, fan, newspap, g...",if fire want fan readili fan newspap govern gr...,175,62,49,7,1,3,...,0,0,0,0,0,0,0,0.035520,0.063073,0.038550
2,id00134,And when they had broken down the frail door t...,"[and, broken, frail, door, found, two, clean, ...",and broken frail door found two clean pick hum...,114,33,30,3,0,1,...,0,0,0,0,1,0,1,0.027486,0.055142,0.057723


In [62]:
# del df_test['cleaned_text']
# del df_test['cleaned_text_string']
del df_test['w2v_array']
del df_train['w2v_array']


In [149]:
cols=(df_train.columns.tolist())[6:]
[item for item in df_train.columns.tolist() if item not in df_test.columns.tolist()]

['author2', 'author', 'cleaned_text', 'cleaned_text_string']

In [183]:
len(cols)

235

In [168]:
cols[0:235]

['length',
 'num_words',
 'num_unique_words',
 'num_punctuations',
 'num_words_upper',
 'num_words_title',
 'mean_word_len',
 'num_stopwords',
 'lexical_diversity',
 'even',
 'seem',
 'friend',
 'thought',
 'hand',
 'hope',
 'fear',
 'mind',
 'ever',
 'place',
 'feel',
 'room',
 'natur',
 'everi',
 'face',
 'happi',
 'love',
 'death',
 'heard',
 'heart',
 'raymond',
 'chang',
 'life',
 'like',
 'still',
 'strang',
 'much',
 'night',
 'two',
 'window',
 'saw',
 'made',
 'men',
 'mani',
 'must',
 'appear',
 'never',
 'thus',
 'dark',
 'know',
 'eye',
 'pass',
 'howev',
 'dream',
 'shall',
 'man',
 'us',
 'long',
 'might',
 'yet',
 'look',
 'light',
 'die',
 'old',
 'may',
 'first',
 'street',
 'say',
 'well',
 'upon',
 'one',
 'father',
 'come',
 'thing',
 'time',
 'word',
 'though',
 'day',
 'hous',
 'return',
 'great',
 'year',
 'littl',
 'whose',
 'came',
 'said',
 'see',
 'door',
 'certain',
 'found',
 'mws_index',
 'eap_index',
 'hpl_index',
 'svd_word_0',
 'svd_word_1',
 'svd_word_

In [182]:
ds_test

array([['id24265',
        'That which is not matter, is not at all unless qualities are things.',
        32, ..., 0.011050695953026251, 0.05808397604522373,
        -0.015135726659374661],
       ['id25917',
        'I sought for repose although I did not hope for forgetfulness; I knew I should be pursued by dreams, but did not dread the frightful one that I really had.',
        84, ..., -0.009014895176013588, -0.010479364908868475,
        0.027816286396748116],
       ['id04951',
        'Upon the fourth day of the assassination, a party of the police came, very unexpectedly, into the house, and proceeded again to make rigorous investigation of the premises.',
        90, ..., -0.034283603615444705, -0.04589236208720345,
        -0.03883216131000478],
       ..., 
       ['id27002', 'Say something about objectivity and subjectivity.', 25,
        ..., 0.00596432580043175, -0.005367215224570375,
        -0.0020850441738909514],
       ['id21418',
        'In each of the five smalle

In [176]:
ds_test=df_test[cols].values

In [179]:
print(ds_test.shape)

(236, 237)


In [180]:
x_t=ds_test[:, :]
y_t=df_test['id'].values
xg_t=xgb.DMatrix(x_t)
pred_prob = bstp.predict(xg_t).reshape(y_t.shape[0], 3)
pred_label = np.argmax(pred_prob, axis=1)

ValueError: could not convert string to float: 'At other times I was quickly and impetuously smitten.'

In [83]:
print(pred_prob)

[[  2.07466558e-02   4.53274138e-03   9.74720597e-01]
 [  9.95783210e-01   2.83912686e-03   1.37771375e-03]
 [  1.08631335e-01   8.89534473e-01   1.83415657e-03]
 ..., 
 [  9.35469210e-01   2.85144355e-02   3.60163823e-02]
 [  1.29575823e-02   2.84086540e-03   9.84201610e-01]
 [  8.50096811e-03   9.90899801e-01   5.99243445e-04]]


In [84]:
export=pd.DataFrame(pred_prob)
export.insert(loc=0, column='id', value=y_t)
export.columns=['id','EAP', 'HPL', 'MWS']
export.head()

,id,EAP,HPL,MWS
0,id02310,0.020747,0.004533,0.974721
1,id24541,0.995783,0.002839,0.001378
2,id00134,0.108631,0.889534,0.001834
3,id27757,0.842110,0.156198,0.001692
4,id04081,0.972890,0.018753,0.008357


In [85]:
len(export)

8392

In [86]:
export[export['id']=='id23301']

,id,EAP,HPL,MWS
6106,id23301,0.175089,0.22535,0.59956


In [87]:
export.to_csv(path_or_buf="../data/export.csv", index=False)

In [ ]:
df_train['']

In [ ]:
# word2vec attempt

import gensim
# let X be a list of tokenized texts (i.e. list of lists of tokens)
model = gensim.models.Word2Vec(filtered_words, size=100)
w2v = dict(zip(model.wv.index2word, model.wv.syn0))